In [1]:
from Utils import *

cohort_root = 'example_patients.csv'

#### First We can load in the example patient dataset into a preprocessed data structure

In [2]:
sd = LNDataset(cohort_root)

#### The Raw input data looks like this
We're storing information on gender and ID For housekeeping purposes
Tm Laterality is telling use which side of the head the main tumor is one (Left, Right, Bilateral)

Affected Lymph Node UPPER is the input where care about.  This gives us a string of the regions in the head with lymph nodes affected by cancer.  E.G. L2, L3 correspond to a patient with nodes in the left side of their head.
Since we only write L2 here, we asusme that 2A, 2B, and 3 are all affected in this patients left side of the head

These remaining values are all used to measure toxicity during a 6 month post-treatment follow up for the patients.

 * Neck Boost: The patient recieved extra radiation to their neck
 * Feeding tube: If a patient needed a feeding tube inserted/Swallowing issues
 * Aspiration: If a patient needed a procedure to remove fluid from their lungs/Breathing issues
 
Aspiration rate pre vs post therapy and overall is usually related, but in this example, we're simply looking at Aspriation rate overall

In [3]:
sd.data.drop('nodes',axis=1).head()

,Gender,Tm Laterality (R/L),Affected Lymph node UPPER,Neck boost (Y/N),Feeding tube 6m,Aspiration rate Pre-therapy,Aspiration rate Post-therapy,Aspiration rate(Y/N)
id,,,,,,,,
1,Male,R,"L1B, L2, L3, R2, R3",N,N,N,Y,N
2,Male,R,"L2, L3",Y,Y,N,N,N
3,Male,R,"L2,R2",Y,N,N,Y,N
4,Female,L,"R2, R3, L2, L3",Y,N,N,Y,N
5,Male,R,"R1B, R2, R3, R4, L2",Y,N,Y,N,Y


For our graph representation, we can one-hot encode the affected nodes for each patient. 
First we consider both the left and right side of the head.

In [4]:
sd.monograms.head()

,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,LRPLN,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
id,,,,,,,,,,,,,,,,,,,,
1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


We can also merge them together to get a dual-graph representation, which we use as the values for our 'nonspatial' baseline representation of the users

In [5]:
sd.dual_monograms.head()

,1A,1B,2A,2B,3,4,5A,5B,6,RPLN
id,,,,,,,,,,
1,0.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0


These nodes are connected, so they form a graph.  We can show this as an adjancency matrix

In [6]:
sd.adjacency

,1A,1B,2A,2B,3,4,5A,5B,6,RPLN
1A,1,1,0,0,0,0,0,0,1,0
1B,1,1,1,0,1,0,0,0,0,0
2A,0,1,1,1,1,0,0,0,0,0
2B,0,0,1,1,0,0,1,0,0,0
3,0,1,1,0,1,1,1,0,1,0
4,0,0,0,0,1,1,0,1,1,0
5B,0,0,0,1,1,0,1,1,0,0
5B,0,0,0,0,0,1,1,1,0,0
6,1,0,0,0,1,1,0,0,1,0
RPLN,0,0,0,0,0,0,0,0,0,1


we can also enumerate all valid node combinations that form an edge

In [7]:
' '.join(sd.get_bigram_names()[0])

'1A1B 1A6 1B2A 1B3 2A2B 2A3 2B5A 34 35A 36 45B 46 5A5B'

Extracting these edges (where an edge is affected if both node are) can be done in the same way as before

In [8]:
sd.dual_bigrams.sample(frac=.02, replace = False)

,1A1B,1A6,1B2A,1B3,2A2B,2A3,2B5A,34,35A,36,45B,46,5A5B
id,,,,,,,,,,,,,
193,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The combination of these form our complete dual graph representation that we do clustering on

Notice that we also have a bilateral value on the end.  This is an extra input equal to the total number of bilaterally affected nodes (will always be a multiple of 2).

In [9]:
sd.spatial().sample(frac=.02, replace = False)

,1A1B,1A6,1B2A,1B3,2A2B,2A3,2B5A,34,35A,36,...,1B,2A,2B,3,4,5A,5B,6,RPLN,bilateral
id,,,,,,,,,,,,,,,,,,,,,
47,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
274,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
78,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
319,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0
82,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,...,0.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0
152,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
##our simpler version without graph edges
sd.nonspatial().sample(frac=.02, replace = False)

,1A,1B,2A,2B,3,4,5A,5B,6,RPLN,bilateral
id,,,,,,,,,,,
130,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
329,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
182,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
180,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
289,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


We can do clustering on these groups.  Here were testing for k = 6, as well as getting clusters of patients that are all almost identical

In [11]:
patterns, clusters, leaves = get_clustering(sd.spatial().values)

In [12]:
print(len(np.unique(patterns)), 'unique patterns')

53 unique patterns


Our final processed data goes into a json file to show on the interface.  The format renames some values, parses the nodes into a nice formated list, and includes the patient similarity scores and cluster results

this final format ends up being matched to a list of key-value paired json objects in a list, one json object per row, with the columns as values.  This is done by 

In [13]:
json_ready_dataset(sd).head()

,gender,position,Affected Lymph node UPPER,Neck_boost,Feeding_tube_6m,Aspiration_rate_Pre-therapy,Aspiration_rate_Post-therapy,Aspiration rate(Y/N),nodes,similarity,scores,id
id,,,,,,,,,,,,
1,Male,R,"L1B, L2, L3, R2, R3",N,N,N,Y,N,"[L1B, L2A, L2B, L3, R2A, R2B, R3]","[1, 111, 335, 356, 243, 199, 259, 266, 185, 27...","[1.0, 1.0, 1.0, 0.4869565217391305, 0.48695652...",1
2,Male,R,"L2, L3",Y,Y,N,N,N,"[L2A, L2B, L3]","[2, 74, 324, 95, 92, 238, 89, 88, 180, 237, 18...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",2
3,Male,R,"L2,R2",Y,N,N,Y,N,"[L2A, L2B, R2A, R2B]","[3, 253, 67, 288, 295, 299, 191, 239, 75, 261,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",3
4,Female,L,"R2, R3, L2, L3",Y,N,N,Y,N,"[R2A, R2B, R3, L2A, L2B, L3]","[4, 356, 56, 57, 86, 131, 140, 168, 20, 185, 1...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",4
5,Male,R,"R1B, R2, R3, R4, L2",Y,N,Y,N,Y,"[R1B, R2A, R2B, R3, R4, L2A, L2B]","[5, 124, 70, 188, 34, 157, 218, 82, 255, 341, ...","[1.0, 1.0, 1.0, 1.0, 0.48529411764705876, 0.47...",5


We can do some analysis on the distribution an p-values of these clusters.  However, given that we randomize this example dataset, we shouldn't see any correlation here

In [14]:
###patient clusters nad toxicity distribution
get_cluster_percentages(sd, clusters)         

,%Aspiration rate(Y/N),%Feeding tube 6m,%Neck boost (Y/N),Aspiration rate(Y/N) count,Feeding tube 6m count,Neck boost (Y/N) count,Total Count
1,16.7,16.7,83.3,1.0,1.0,5.0,6.0
2,15.2,16.8,62.4,19.0,21.0,78.0,125.0
3,13.0,17.1,67.9,25.0,33.0,131.0,193.0
4,28.6,9.5,57.1,6.0,2.0,12.0,21.0
5,33.3,0.0,100.0,1.0,0.0,3.0,3.0
6,37.5,12.5,87.5,3.0,1.0,7.0,8.0


In [15]:
#p-value using fisher's exact test for correlation with clusters and outcomes
get_cluster_correlations(sd, clusters)

Feeding tube 6m         0.975013
Aspiration rate(Y/N)    0.106699
Neck boost (Y/N)        0.413958
Name: p-value, dtype: float64